<a href="https://colab.research.google.com/github/slazur83/Tableau/blob/main/Sport%20Activity/Archived/Workouts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gpxpy
!pip install xlwt

import os
import pandas as pd
import numpy as np
import json
import gpxpy
from datetime import datetime
from dateutil import parser

workouts_folder_path1 = "/content/drive/MyDrive/Dane z aplikacji/Endomondo/Workouts"
workouts_folder_path2 = "/content/drive/MyDrive/Dane z aplikacji/MapMyRun/user184238914_workout_history.csv"
workouts_folder_path3 = "/content/drive/MyDrive/Dane z aplikacji/Sports Tracker/Running workouts"
workouts_folder_path4 = "/content/drive/MyDrive/Dane z aplikacji/Garmin Connect/Aktywności/Bieganie.csv"
google_sheets_link_1 = 'https://docs.google.com/spreadsheets/d/1I6XjfT990f6rArjYedIf0nw5vsBye_L6vkMILD8YjZI/edit?usp=sharing'
google_sheets_link_2 = 'https://docs.google.com/spreadsheets/d/1sKQRBWY5xvY-erhFxXo0bC0pTpZT6QUUMx1fjLngGk0/edit?usp=drive_link'

dfs = []

# Endomondo JSON
files = [f for f in os.listdir(workouts_folder_path1) if f.endswith(".json")]
if files:
    for file_name in files:
        file_path = os.path.join(workouts_folder_path1, file_name)
        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
                if isinstance(data, list) and len(data) > 0:
                    sport = 'Running' if data[0].get('sport') == 'RUNNING' else data[0].get('sport')
                    if sport == 'Running':
                        source = data[1].get('source')
                        source = source.replace("INPUT_MANUAL", "Endomondo (Manual)").replace("TRACK_MOBILE", "Endomondo (GPS)") if pd.notna(source) else 'Endomondo'
                        start_time = pd.to_datetime(data[3].get('start_time'))
                        end_time = pd.to_datetime(data[4].get('end_time'))
                        speed_avg_kmh = round(float(data[8].get('speed_avg_kmh')), 2) if data[8].get('speed_avg_kmh') is not None else None
                        duration = (str(data[5].get('duration_s')).split('.')[0]) if data[5].get('duration_s') is not None else None
                        distance = round(float(data[6].get('distance_km')), 2) if duration is not None else None

                        dfs.append({
                            'sport': sport,
                            'source': source,
                            'date': start_time.date(),
                            'start_time': start_time.time(),
                            'end_time': end_time.time(),
                            'duration_s': duration,
                            'distance_km': distance,
                            'speed_avg_kmh': speed_avg_kmh
                        })
            except Exception as e:
                print(f"Error processing JSON file {file_name}: {e}")

    df1 = pd.DataFrame(dfs).sort_values(by='date').reset_index(drop=True)
    df1['date'] = pd.to_datetime(df1['date'].astype(str).str.strip())
    df1['distance_km'] = df1['distance_km'].astype(float).round(2)
    df1['duration_s'] = pd.to_numeric(df1['duration_s'], errors='coerce').astype('Int64')
else:
    print("No JSON files found in the specified folder.")

# MapMyRun CSV
df = pd.read_csv(workouts_folder_path2, header=None, skip_blank_lines=False)
df2 = df[df[2] == 'Run'].copy()
df2.rename(columns={1: 'date', 2: 'sport', 4: 'distance_km', 5: 'duration_s', 8: 'speed_avg_kmh', 13: 'source2', 14:'url'}, inplace=True)
df2['sport'] = 'Running'
df2['source'] = 'Map My Run app'
df2.drop(columns=[0, 3, 6, 7, 9, 10, 11, 12], inplace=True)

df2['date'] = pd.to_datetime(df2['date'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d')))
df2['distance_km'] = df2['distance_km'].astype(float).round(2)
df2['duration_s'] = df2['duration_s'].astype(int)
df2['speed_avg_kmh'] = df2['speed_avg_kmh'].astype(float).round(2)
df2 = df2[['sport', 'source', 'date', 'duration_s', 'distance_km', 'speed_avg_kmh']]

# Sports Tracker GPX
gpx_data_list = []
for filename in os.listdir(workouts_folder_path3):
    if filename.endswith(".gpx"):
        gpx_file_path = os.path.join(workouts_folder_path3, filename)
        with open(gpx_file_path, "r") as gpx_file:
            gpx_data = gpxpy.parse(gpx_file)
            if gpx_data.tracks and gpx_data.tracks[0].segments and gpx_data.tracks[0].segments[0].points:
                start_time = gpx_data.tracks[0].segments[0].points[0].time
                date = start_time.strftime("%Y-%m-%d")
                start_time = start_time.strftime("%H:%M:%S")
                end_time = gpx_data.tracks[0].segments[0].points[-1].time
                duration_seconds = int((end_time - gpx_data.tracks[0].segments[0].points[0].time).total_seconds())
                distance_km = round((gpx_data.length_3d() / 1000), 2)
                speed_avg_kmh = round(distance_km / (duration_seconds / 3600), 2)

                gpx_data_list.append({
                    "sport": "Running",
                    "source": "Sports Tracker",
                    "title": np.nan,
                    "date": date,
                    "start_time": start_time,
                    "duration_s": duration_seconds,
                    "distance_km": distance_km,
                    "speed_avg_kmh": speed_avg_kmh
                })

df3 = pd.DataFrame(gpx_data_list)
df3['date'] = pd.to_datetime(df3['date'])

# Garmin Connect CSV
def convert_pace_to_speed(pace):
    pace_in_seconds = int(pace.split(':')[0]) * 60 + int(pace.split(':')[1])
    speed_in_kph = 3600 / pace_in_seconds
    return speed_in_kph

df4 = pd.read_csv(workouts_folder_path4, skip_blank_lines=False)
df4['source'] = 'Garmin Connect'
df4['Typ aktywności'] = df4['Typ aktywności'].replace('Bieganie', 'Running')
df4['Czas'] = pd.to_timedelta(df4['Czas']).dt.total_seconds().astype('int64')
df4['Data'] = pd.to_datetime(df4['Data'])
df4['start_time'] = df4['Data'].dt.time
df4.rename(columns={'Typ aktywności': 'sport', 'Data': 'date', 'Czas': 'duration_s',
                    'Dystans': 'distance_km', 'Średnie tempo': 'speed_avg_kmh', 'Tytuł': 'title'}, inplace=True)
df4['speed_avg_kmh'] = df4['speed_avg_kmh'].apply(convert_pace_to_speed).round(2)
df4['title'] = df4['title'].apply(lambda x: np.nan if 'Bieganie' in str(x) else x)
df4 = df4[['sport', 'source', 'title', 'date', 'start_time', 'duration_s', 'distance_km', 'speed_avg_kmh']]

# Merge Endomondo + MapMyRun
merged_df = pd.merge(df2, df1, on=['date', 'distance_km'], how='left')
df12 = pd.DataFrame()
df12['sport'] = merged_df['sport_x']
df12['source'] = merged_df['source_x'].combine_first(merged_df['source_y'])
df12['date'] = merged_df['date']
df12['start_time'] = merged_df.get('start_time')
df12['end_time'] = merged_df.get('end_time')
df12['distance_km'] = merged_df['distance_km']
df12['duration_s'] = merged_df['duration_s_x'].combine_first(merged_df['duration_s_y'])
df12['speed_avg_kmh'] = merged_df['speed_avg_kmh_x'].combine_first(merged_df['speed_avg_kmh_y'])
df12['title'] = np.nan

# Combine all sources
df123 = pd.concat([df12, df3], ignore_index=True)
df123 = df123[df123['source'] != 'Sports Tracker']
df12345 = pd.concat([df123, df4], ignore_index=True)
df12345['type'] = df12345['title'].apply(lambda x: 'Workout' if pd.isna(x) else 'Sport event')
df12345 = df12345[['source', 'sport', 'type', 'title', 'date', 'start_time', 'distance_km', 'duration_s', 'speed_avg_kmh']]
df12345 = df12345.sort_values(by='date')

# Google Sheets Integration
document_key = google_sheets_link_1.split('/')[-2]
worksheet_link = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv'
dfgoogle_1 = pd.read_csv(worksheet_link, header=1)
dfgoogle_1['data'] = pd.to_datetime(dfgoogle_1['Data'], dayfirst=False)

df_nans = df12345.loc[df12345['source'].isna()]
merged_df = pd.merge(df_nans, dfgoogle_1[['data', 'Nazwa zawodów']], left_on='date', right_on='data', how='inner')
merged_df['title'] = merged_df['Nazwa zawodów']
merged_df['source'] = "Google Sheets"
merged_df['type'] = "Sport event"
merged_df = merged_df.drop(['data', 'Nazwa zawodów'], axis=1)

# Final dataframe
dates_to_remove = merged_df['date'].tolist()
df_filtered = df12345[~((df12345['date'].isin(dates_to_remove)) & (df12345['source'].isna()))]
final = pd.concat([df_filtered, merged_df], ignore_index=True)

# Save Excel as .xlsx
excel_file_path = '/content/drive/MyDrive/Moje projekty/Tableau/Source data/workouts.xlsx'
final.to_excel(excel_file_path, index=False)
print("Saved final dataframe to", excel_file_path)
